# Agenda
Classification of GPs' bitmaps.



In [1]:
# Installing Libraries (if not installed)
#!pip3 install jupyterlab-git
#!pip3 install voila
#!pip3 install matplotlib
#!pip3 install scikit-learn
#!pip3 install seaborn
#!pip3 install numpy
#!pip3 install pandas
#!pip3 install so4gp
#!pip3 install scipy
#!pip3 install sympy
#!pip3 install networkx

# Import libraries <a name="lib-import"></a>

In [1]:
# Importing Libraries
import pandas as pd
import numpy as np
import scipy
import math

#from numpy import linalg as la
import itertools
import pylab as pl

from sklearn import svm, linear_model#, cross_validation

In [2]:
import so4gp as sgp

# Data set
Retrieving data set and the features

In [4]:
ds = sgp.DataGP('../data/DATASET.csv')
#ds = sgp.DataGP('../data/c2k_02k.csv')
#ds = sgp.DataGP('../data/breast_cancer.csv')


data_size = ds.row_count
feature_size = ds.col_count
features = np.array(ds.data, dtype=np.float64)
features

array([[30.,  3.,  1., 10.,  2.,  4.],
       [35.,  2.,  2.,  8.,  2.,  4.],
       [40.,  4.,  2.,  7.,  2.,  4.],
       [50.,  1.,  1.,  6.,  2.,  4.],
       [52.,  7.,  1.,  2.,  2.,  4.]])

In [5]:
sgp.graank('../data/breast_cancer.csv', min_sup=0.6)

'{"Algorithm": "GRAANK", "Patterns": [[["Insulin+", "BMI+"], 0.612], [["BMI+", "HOMA+"], 0.614], [["BMI+", "Leptin+"], 0.732], [["Insulin-", "BMI-"], 0.612], [["HOMA-", "BMI-"], 0.614], [["Leptin-", "BMI-"], 0.732], [["Insulin+", "Glucose+"], 0.637], [["HOMA+", "Glucose+"], 0.697], [["Glucose-", "Insulin-"], 0.637], [["HOMA-", "Glucose-"], 0.697], [["Insulin+", "HOMA+"], 0.94], [["Insulin+", "Leptin+"], 0.659], [["HOMA-", "Insulin-"], 0.94], [["Leptin-", "Insulin-"], 0.659], [["HOMA+", "Leptin+"], 0.659], [["HOMA-", "Leptin-"], 0.659], [["MCP.1+", "Resistin+"], 0.672], [["Resistin-", "MCP.1-"], 0.672], [["Insulin+", "HOMA+", "Glucose+"], 0.637], [["HOMA-", "Glucose-", "Insulin-"], 0.637], [["Insulin+", "HOMA+", "Leptin+"], 0.629], [["HOMA-", "Leptin-", "Insulin-"], 0.629]]}'

In [3]:
feature_X = features[:, [1,3]]
feature_X

array([[ 3., 10.],
       [ 2.,  8.],
       [ 4.,  7.],
       [ 1.,  6.],
       [ 7.,  2.]])

# Pairwise Clustering

According to the work of R. Srikant, it is possible to cluster users based on incomplete information on their purchasing preferences. They achieve this by converting an (Mx2) matrix about the choices of users over products into an (M) 'net wins matrix'.

For instance assume we have 4 products (r1, r2, r3, r4) and 3 users (u1, u2, u3). We use the following encoding: if a user picks r1 over r2 (r1 > r2) then (r1,r2)=1 and (r2,r1)=-1; if we do not have a preferred choice (this accounts for incomplete information) between r3 and r4 then the encoding is (r3,r4)=0. 

